In [1]:
import torch

from dataset import RawDataset
from transducer import Transducer
from settings import make_settings
from sklearn.model_selection import train_test_split

## Define Experiment settings

In [2]:
name = "cmudict"

In [3]:
settings = make_settings(
    epochs=1, batch=16, device=torch.device('cuda:0'), scheduler="exponential", gamma=1.0,
    verbose=True, report_progress_every=10, main_metric="edit_distance", keep_only_best_checkpoint=True,
    use_features=False, optimizer="adam", lr=0.001, weight_decay=0.0001, grad_clip=None, model="lstm",
    embedding_size=128, hidden_size=256, hidden_layers=1, dropout=0.1, autoregressive=True, tau=5,
    scorer="softmax", temperature=1.0, features_num_layers=1, features_pooling="mlp",
    noop_discount=1.0, allow_copy=True, enforce_copy=True, name=name,
    train_data_path=None, dev_data_path=None, save_path=f"./saved_models/{name}/",
    beam_search=True, num_beams=5, max_decoding_length=40
)

## Load Data

In [4]:
sources, targets = [], []

with open("data/cmudict/cmudict.txt") as cmuf:
    for line in cmuf:
        line = line.strip()
        if line:
            source, target = line.split("  ")
            sources.append(list(source))
            targets.append(target.split())
            
train_sources, test_sources, train_targets, test_targets = train_test_split(
    sources, targets, test_size=0.2, random_state=0, shuffle=True
)
dev_sources, test_sources, dev_targets, test_targets = train_test_split(
    test_sources, test_targets, test_size=0.5, random_state=1, shuffle=True
)


train_data = RawDataset(sources=train_sources, targets=train_targets, features=None)
dev_data = RawDataset(sources=dev_sources, targets=dev_targets, features=None)

## Train Transducer Model

In [5]:
from trainer import load_model

In [6]:
transducer1 = Transducer(settings=settings).fit(train_data=train_data, development_data=dev_data)
predictions1 = transducer1.predict(sources=test_sources)

[08/17/22 15:57:28] INFO     Prepare for Training                              ]8;id=125417;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=818460;file:///home/leander/projects/copy-ctc/trainer.py#310\310]8;;\

                    INFO     Build vocabulary and datasets                     ]8;id=946419;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=770115;file:///home/leander/projects/copy-ctc/trainer.py#311\311]8;;\

[08/17/22 15:57:29] INFO     Train data contains 107027 datapoints             ]8;id=357369;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=270901;file:///home/leander/projects/copy-ctc/trainer.py#320\320]8;;\

                    INFO     Dev data contains 13378 datapoints                ]8;id=561145;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=966744;file:///home/leander/projects/copy-ctc/trainer.py#322\322]8;;\

                    INFO     Source vocabulary contains 48 items               ]8;id=794725;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=103348;file:///home/leander/projects/copy-ctc/trainer.py#323\323]8;;\

...

[08/17/22 16:12:20] INFO     [99.85%] Loss: 2.408 || LR: 0.001000 || Step 6680 ]8;id=364288;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=11357;file:///home/leander/projects/copy-ctc/trainer.py#406\406]8;;\
                             / 6690                                                          

[08/17/22 16:12:21] INFO     [100.00%] Loss: 2.354 || LR: 0.001000 || Step     ]8;id=941468;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=2754;file:///home/leander/projects/copy-ctc/trainer.py#406\406]8;;\
                             6690 / 6690                                                     

[08/17/22 16:13:18] INFO     [Development metrics]    Loss: 2.3150 || WER:     ]8;id=349151;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=932813;file:///home/leander/projects/copy-ctc/trainer.py#426\426]8;;\
                             49.84 || Edit-Distance: 0.89                                    

                    INFO     Saving Model after epoch 1                        ]8;id=23437;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=278378;file:///home/leander/projects/copy-ctc/trainer.py#455\455]8;;\

Prediction Progress: 100%|█████████████████████████████████████████████| 837/837 [40:42<00:00,  2.92s/it]


In [7]:
transducer2 = Transducer(settings=settings).fit(train_data=train_data, development_data=dev_data)
predictions2 = transducer2.predict(sources=test_sources)

[08/17/22 16:54:00] INFO     Prepare for Training                              ]8;id=546714;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=409400;file:///home/leander/projects/copy-ctc/trainer.py#310\310]8;;\

                    INFO     Build vocabulary and datasets                     ]8;id=485089;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=404113;file:///home/leander/projects/copy-ctc/trainer.py#311\311]8;;\

[08/17/22 16:54:01] INFO     Train data contains 107027 datapoints             ]8;id=380388;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=537416;file:///home/leander/projects/copy-ctc/trainer.py#320\320]8;;\

                    INFO     Dev data contains 13378 datapoints                ]8;id=997344;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=199618;file:///home/leander/projects/copy-ctc/trainer.py#322\322]8;;\

                    INFO     Source vocabulary contains 48 items               ]8;id=49477;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=970735;file:///home/leander/projects/copy-ctc/trainer.py#323\323]8;;\

...

[08/17/22 17:08:22] INFO     [100.00%] Loss: 2.525 || LR: 0.001000 || Step     ]8;id=355854;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=982235;file:///home/leander/projects/copy-ctc/trainer.py#406\406]8;;\
                             6690 / 6690                                                     

[08/17/22 17:09:19] INFO     [Development metrics]    Loss: 2.2828 || WER:     ]8;id=666149;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=137560;file:///home/leander/projects/copy-ctc/trainer.py#426\426]8;;\
                             49.73 || Edit-Distance: 0.88                                    

                    INFO     Saving Model after epoch 1                        ]8;id=56465;file:///home/leander/projects/copy-ctc/trainer.py\trainer.py]8;;\:]8;id=557877;file:///home/leander/projects/copy-ctc/trainer.py#455\455]8;;\

Prediction Progress:  53%|███████████████████████▉                     | 446/837 [35:10<30:50,  4.73s/it]


KeyboardInterrupt: 

In [ ]:
from ensemble import ensemble_predict

In [ ]:
ensemble_predictions, _ = ensemble_predict([transducer1, transducer2], sources=test_sources, features=None)

In [ ]:
import numpy as np

In [ ]:
ensemble_predictions_cleaned = [
    [
        symbol for symbol in prediction.prediction
        if symbol not in transducer1.model.target_vocabulary.get_special_symbols()
    ]
    for prediction in ensemble_predictions
]

In [ ]:
ensemble_predictions_cleaned[0]

In [ ]:
accuracy = np.mean(
    [prediction == target for prediction, target in zip(ensemble_predictions_cleaned, test_targets)]
)
wer = 100 * (1 - accuracy)

print(f"WER: {wer:.2f}")

In [ ]:
predictions2_cleaned = [
    [
        symbol for symbol in prediction.prediction
        if symbol not in transducer1.model.target_vocabulary.get_special_symbols()
    ]
    for prediction in predictions2
]

In [ ]:
accuracy = np.mean(
    [prediction == target for prediction, target in zip(predictions2_cleaned, test_targets)]
)
wer = 100 * (1 - accuracy)

print(f"WER: {wer:.2f}")